In [20]:
import numpy as np
from datetime import timedelta
from dateutil import parser
import pandas as pd
import os

In [2]:
def make_data(data_type):
    df = pd.read_csv('{}/EVO_Training_Data_Original.csv'.format(data_type), index_col=False)

    time = df.apply(lambda x: str(int(x['date '])) +' ' +str(int(x['H'])) + ':' +str(int(x['M'])) + ':' + str(int(x['S'])), axis=1)
    datetime = time.apply(parser.parse)
    df['datetime'] = datetime

    row_list = list()
    step_size = 5
    skip_num = 0
    for i in range(len(df) - step_size):    
        skip_time = False
        tmp = df['datetime'].iloc[i:i+step_size] 
        cap_nom = df['cap_nom'].iloc[i:i+step_size]
        first = tmp.iloc[0]
        if len(cap_nom.unique()) == 1:
            for idx, j in tmp.iteritems():            
                if j - first > timedelta( minutes=12):                
                    skip_num +=1
                    skip_time = True
                    break
                else:
                    first = j
            if not skip_time:
                row_list.append(df.iloc[i:i+step_size])
    print('skipped :', skip_num)

    columns = ['mdot', 'Tod', 'RHod', 'mode', 'Tid', 'Vidu', 'cap_nom', 'Qsens', 'Qlat']
    data = np.zeros((len(row_list * step_size), len(columns)))
    for idx, value in enumerate(row_list):
        data[idx * (step_size) :(idx + 1)* (step_size)] = value[columns].values    

    data = data.reshape(-1, step_size, len(columns))

    target_columns = ['Load_s', 'Load_l']
    target = np.zeros((len(row_list) * step_size, len(target_columns)))
    for idx, value in enumerate(row_list):
        target[idx * (step_size) :(idx + 1)* (step_size)] = value[target_columns].values

    target = target.reshape(-1, step_size, len(target_columns))

    target = target[:, step_size - 1, :]

    np.savez('{}/data'.format(data_type), X=data, y=target)

In [5]:
make_data('Site_1')

skipped : 8597


In [6]:
make_data('Site_5')

skipped : 4


In [4]:
data = np.load('data/Site_1/data.npz')

In [24]:
for test_site in [1, 23, 24, 25]:
    tmp_X = None
    tmp_y = None
    for trn_site in [1, 23, 24, 25]:
        if trn_site == test_site:
            continue
        data = np.load('data/Site_{}/data.npz'.format(test_site))
        if tmp_X is None:            
            tmp_X = data['X']
            tmp_y = data['y']
            print('      ', tmp_X.shape)
        else:
            print('           ', tmp_X.shape, data['X'].shape)
            tmp_X = np.concatenate((tmp_X, data['X']), 0)
            tmp_y = np.concatenate((tmp_y, data['y']), 0)
            
        
    os.makedirs('data/merged_except_{}'.format(test_site))
    np.savez('data/merged_except_{}/data'.format(test_site), X=tmp_X, y=tmp_y)

       (18743, 5, 9)
            (18743, 5, 9) (18743, 5, 9)
            (37486, 5, 9) (18743, 5, 9)
       (49759, 5, 9)
            (49759, 5, 9) (49759, 5, 9)
            (99518, 5, 9) (49759, 5, 9)
       (42375, 5, 9)
            (42375, 5, 9) (42375, 5, 9)
            (84750, 5, 9) (42375, 5, 9)
       (31690, 5, 9)
            (31690, 5, 9) (31690, 5, 9)
            (63380, 5, 9) (31690, 5, 9)


In [13]:
tmp_X.shape

(18743, 5, 9)